<a href="https://colab.research.google.com/github/thesamedorcas/Credit-Card-Fraud-Detection/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'creditcardfraud:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F310%2F23498%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240812%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240812T141830Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D600cb1a529f03ef2fbb73438e2114d637139eefc4e4b6c30eaddda154dc23757c3665e4fa4ba3f764567b41654d9bc0e65d273972682fb79eaa35cd09a48331640a9403fd8c53949c9467bd7c22b32ccdce66dbadfde74299721ec560b85a2fa9ff355d8825648ff9f2e310b3a8f279bb21050f07ba8489acaa99ff4f15d784c995ce8c193666aee792a6d6517e84335129cb152a69a2fd0c07bf7ada459eebcbd78c8065f0a47870bdbd6c3ccaead27536ee8917385f0a04274210bc8058530c03137d654a1da4496f5e30a7f61afdf23b6bfefce8fcb4ad55bb8d90e04a2709c01014169047f9162530d0b3bb4c2158a967e01ac8d29744468dfa3e0a4d940'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
#Importing the dependecies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
#Loading the dataset to the Pandas dataframe
credit_card_data= pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

In [ ]:
#I want to see the first 5 rows of the dataset to see what features the data set has
credit_card_data.head()

In [ ]:
#Last 5 rows
credit_card_data.tail()


In [ ]:
#Gathering dataset information
credit_card_data.info()

In [ ]:
#checking the number of missing values in each column
credit_card_data.isnull().sum()
#No missing values

In [ ]:
#Distribution of legit transactions and fraudulent transaction( 0- legit, 1- fraudulent)
credit_card_data['Class'].value_counts()

This dataset is highly unbalanced

0- legit
1-fraudulent


In [ ]:
legit= credit_card_data[credit_card_data.Class ==0]
fraud= credit_card_data[credit_card_data.Class==1]


In [ ]:
print(legit.shape)
print(fraud.shape)

In [ ]:
#statistical measures of the data
legit.Amount.describe()

In [ ]:
fraud.Amount.describe()

In [ ]:
#compare values for btoh fraudulent and legit transactions
credit_card_data.groupby("Class").mean()

Under-sampling
Build sample dataset containibg similar distribution of normal transaction and the fraudulent transactions
No of fraud transactions- 492
I'm going to take 492 random samples from the legit transactions so I can compare it to the fraudulent transactions therefore making it event and easy to differentiate

In [ ]:
legit_sample= legit.sample(n=492)

Concatenating 2 ddata frames- the legit and fraudulent

In [ ]:
new_dataset= pd.concat([legit_sample, fraud], axis= 0)

In [ ]:
new_dataset.head()

In [ ]:
new_dataset.tail()

In [ ]:
new_dataset["Class"].value_counts()

In [ ]:
new_dataset.groupby("Class").mean()

Splitting the data into features and targets

In [ ]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [ ]:
print(X)

In [ ]:
print(Y)

Split data into training and testing data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

Model Training


In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
#Training the logistic Regression model with training data
model.fit(X_train, Y_train)


Model Evaluation
Accuracy Score

In [ ]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print("Accuracy on training data: ", training_data_accuracy)

In [ ]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print('Accuracy score on Test Data : ', test_data_accuracy)